# Benchmarking for Enumeration

In [ ]:
import timeit
from itertools import product
import tqdm

import pandas as pd

from flippy import flip, condition, mem, draw_from
from flippy.callentryexit import register_call_entryexit, EnterCallState, ExitCallState
from flippy.map import independent_map
from flippy.interpreter import CPSInterpreter
from flippy.core import ReturnState
from flippy.inference.enumeration import Enumeration
from flippy.inference.simpleenumeration import SimpleEnumeration

def model_with_map():
    def f(p):
        return flip(p) + flip(p)
    x = independent_map(f, (0.2, 0.2, 0.5, 0.5, 0.5, .7))
    return sum(x)

def model_without_map():
    def f(p):
        return flip(p) + flip(p)
    x = (f(0.2), f(0.2), f(0.5), f(0.5), f(0.5), f(.7))
    return sum(x)


In [ ]:
number = 20
params_space = dict(
    function=[model_with_map, model_without_map],
    _call_cache_size=[0, 2, 1000],
    _map_cross_product=[True, False],
    _enumeration_strategy=['tree', 'graph'],
)

Enumeration_results = []
param_names, param_values = zip(*params_space.items())
for param_assn in tqdm.tqdm(list(product(*param_values))):
    params = dict(zip(param_names, param_assn))
    t = timeit.timeit(
        lambda : Enumeration(**params).run(),
        number=number
    )
    Enumeration_results.append({
        **params,
        'function': params['function'].__name__,
        'time_seconds': t/number
    })

100%|██████████| 24/24 [03:01<00:00,  7.56s/it]


In [ ]:
pd.DataFrame(Enumeration_results)

,function,_call_cache_size,_map_cross_product,_enumeration_strategy,time_seconds
0,model_with_map,0,True,tree,0.053023
1,model_with_map,0,True,graph,0.044312
2,model_with_map,0,False,tree,0.638201
3,model_with_map,0,False,graph,0.344143
4,model_with_map,2,True,tree,0.050053
5,model_with_map,2,True,graph,0.034992
6,model_with_map,2,False,tree,0.643110
7,model_with_map,2,False,graph,0.351781
8,model_with_map,1000,True,tree,0.050284
9,model_with_map,1000,True,graph,0.035991
